In [4]:
import pandas as pd
import numpy as np
import os

def transform_values(input_excel_path, output_csv_path=None):
    """
    Read Excel file with original values and target points,
    apply piecewise linear transformation, and save results.
    
    Parameters:
    -----------
    input_excel_path : str
        Path to input Excel file
    output_csv_path : str, optional
        Path to save output CSV file (if None, won't save CSV)
    
    Returns:
    --------
    DataFrame with original and corrected values
    """
    print(f"Reading Excel file: {input_excel_path}")
    
    # Read the Excel file
    df = pd.read_csv(input_excel_path)
    
    # Check if required columns exist
    if 'Chainage' not in df.columns:
        raise ValueError("Column 'Chainage' not found in the input file")
    if 'Reference' not in df.columns:
        raise ValueError("Column 'Reference' not found in the input file")
    
    # Convert columns to numeric, replacing non-numeric values with NaN
    df['Chainage'] = pd.to_numeric(df['Chainage'], errors='coerce')
    df['Reference'] = pd.to_numeric(df['Reference'], errors='coerce')
    
    # Extract original values from Chainage column
    original_values = df['Chainage'].dropna().values
    
    # Extract anchor points (rows with values in both Chainage and Reference columns)
    anchor_mask = df['Chainage'].notna() & df['Reference'].notna()
    anchor_points = df[anchor_mask][['Chainage', 'Reference']].values
    
    # Sort anchor points by Chainage value
    anchor_points = anchor_points[anchor_points[:, 0].argsort()]
    
    print(f"Found {len(original_values)} original values")
    print(f"Found {len(anchor_points)} anchor points")
    
    if len(anchor_points) < 2:
        raise ValueError("Need at least 2 anchor points for transformation")
    
    # Print anchor points for debugging
    print("\nAnchor points:")
    for i, (orig, target) in enumerate(anchor_points):
        print(f"  Point {i+1}: Chainage={orig} -> Reference={target}")
    
    def apply_transformation(value):
        """Apply piecewise linear transformation to a single value."""
        # Check if value exactly matches an anchor point
        for orig, target in anchor_points:
            if abs(value - orig) < 1e-6:
                return target
                
        # Find which segment the value belongs to
        segment_idx = np.searchsorted(anchor_points[:, 0], value)
        
        # Handle values outside defined anchor points
        if segment_idx == 0:
            # Before first anchor point - extrapolate using first segment
            p1_orig, p1_target = anchor_points[0]
            p2_orig, p2_target = anchor_points[1]
            slope = (p2_target - p1_target) / (p2_orig - p1_orig)
            return p1_target - slope * (p1_orig - value)
            
        elif segment_idx == len(anchor_points):
            # After last anchor point - extrapolate using last segment
            p1_orig, p1_target = anchor_points[-2]
            p2_orig, p2_target = anchor_points[-1]
            slope = (p2_target - p1_target) / (p2_orig - p1_orig)
            return p2_target + slope * (value - p2_orig)
            
        else:
            # Within defined segments - interpolate
            p1_orig, p1_target = anchor_points[segment_idx-1]
            p2_orig, p2_target = anchor_points[segment_idx]
            
            # Calculate proportion within segment
            proportion = (value - p1_orig) / (p2_orig - p1_orig)
            
            # Apply proportion to target segment
            return p1_target + proportion * (p2_target - p1_target)
    
    # Apply transformation to all values from Chainage column
    corrected_values = df['Chainage'].apply(lambda x: apply_transformation(x) if pd.notna(x) else np.nan)
    
    # Create output DataFrame with all original columns plus corrected values
    result_df = df.copy()
    result_df['Corrected_Value'] = corrected_values
    
    # Calculate statistics (only for non-null values)
    original_values_clean = df['Chainage'].dropna().values
    corrected_values_clean = corrected_values.dropna().values
    
    original_mean = original_values_clean.mean()
    corrected_mean = corrected_values_clean.mean()
    
    print("\nStatistics:")
    print(f"Original chainage - Mean: {original_mean:.4f}, Min: {original_values_clean.min():.4f}, Max: {original_values_clean.max():.4f}")
    print(f"Corrected chainage - Mean: {corrected_mean:.4f}, Min: {corrected_values_clean.min():.4f}, Max: {corrected_values_clean.max():.4f}")
    
    # Save to CSV if path provided
    if output_csv_path:
        result_df.to_csv(output_csv_path, index=False)
        print(f"Saved CSV file: {output_csv_path}")
        
    return result_df


if __name__ == "__main__":
    # Input file path
    input_file = "/home/saran/Documents/Db_data/Invision_V_site2/chainage_correction/rhs - rhs.csv (copy).csv"
    
    # Output file path
    output_csv = input_file.replace(".csv","_output.csv")
    
    # Process the file
    result = transform_values(input_file, output_csv)
    
    print("\nDone!")

Reading Excel file: /home/saran/Documents/Db_data/Invision_V_site2/chainage_correction/rhs - rhs.csv (copy).csv
Found 1778 original values
Found 134 anchor points

Anchor points:
  Point 1: Chainage=1.53 -> Reference=0.0
  Point 2: Chainage=2.027 -> Reference=0.5
  Point 3: Chainage=2.529 -> Reference=1.0
  Point 4: Chainage=3.021 -> Reference=1.5
  Point 5: Chainage=3.526 -> Reference=2.0
  Point 6: Chainage=4.027 -> Reference=2.5
  Point 7: Chainage=4.524 -> Reference=3.0
  Point 8: Chainage=5.034 -> Reference=3.5
  Point 9: Chainage=5.521 -> Reference=4.0
  Point 10: Chainage=6.025 -> Reference=4.5
  Point 11: Chainage=6.535 -> Reference=5.0
  Point 12: Chainage=7.04 -> Reference=5.5
  Point 13: Chainage=7.525 -> Reference=6.0
  Point 14: Chainage=8.035 -> Reference=6.5
  Point 15: Chainage=8.53 -> Reference=7.0
  Point 16: Chainage=9.025 -> Reference=7.5
  Point 17: Chainage=9.519 -> Reference=8.0
  Point 18: Chainage=10.021 -> Reference=8.5
  Point 19: Chainage=10.52 -> Reference=

In [5]:
import mysql.connector
import pandas as pd

sql="Update Motorway.tbl_site_asset set Chainage=%s where id=%s"
conx=mysql.connector.connect(host='takeleap.in',user='seekright',password='Takeleap@123',port='3307')
# conx= mysql.connector.connect(host='seekright-db.ce3lsmnwzkln.ap-south-1.rds.amazonaws.com',user='admin',password='BXWUCSpjRxEqzxXYTF9e',port='3306') 
cur=conx.cursor()
df=pd.read_csv(output_csv)
# comment="Updated with metadata"
for i in range(len(df['row_id'])):
    print(df['Corrected_Value'][i],df['id'][i])
    try:
        cur.execute(sql,(df['Corrected_Value'][i],int(df['id'][i])))
    except Exception as e:
        print("error",e)
conx.commit()
conx.close()

0.0 123226.0
0.0040241448692152 122926.0
0.0130784708249495 123225.0
0.0362173038229376 123602.0
0.0462776659959758 123224.0
0.0472837022132796 124860.0
0.0513078470824948 124140.0
0.0533199195171025 122925.0
0.0754527162977866 123601.0
0.1207243460764586 123600.0
0.1670020120724345 123599.0
0.1811770623742455 124724.0
0.1851106639839033 123223.0
0.2112676056338027 123598.0
0.2474849094567404 123222.0
0.255533199195171 123597.0
0.2665995975855129 124859.0
0.3044768611670018 124734.0
0.3430583501006035 123270.0
0.3440643863179075 123810.0
0.3447786720321931 124723.0
0.3460764587525151 123221.0
0.3661971830985913 124123.0
0.3953722334004023 124122.0
0.4164989939637825 124121.0
0.436619718309859 124120.0
0.4607645875251507 124119.0
0.4869215291750499 124118.0
0.4879275653923541 122924.0
0.5 123220.0
0.5109561752988044 124117.0
0.5225099601593622 124730.0
0.5368525896414342 124116.0
0.5498007968127488 123219.0
0.5498007968127488 124115.0
0.5687250996015936 124114.0
0.5846613545816733 12411